In [2]:
import pandas as pd
import numpy as np

In [3]:
# !wget https://hr-projects-assets-prod.s3.amazonaws.com/lrpqaeb14h/e186636d099c5d032901dfaf5db68852/matches.csv
# !wget https://hr-projects-assets-prod.s3.amazonaws.com/lrpqaeb14h/6f3085efd6250aa3c886059677871a29/deliveries.csv

In [4]:
matches=pd.read_csv("matches.csv")
deliveries=pd.read_csv("deliveries.csv")

In [5]:
batsman_grp = deliveries.groupby(["match_id", "inning", "batting_team", "batsman"])
batsmen = batsman_grp["batsman_runs"].sum().reset_index()

# Ignore the wide balls.
balls_faced = deliveries[deliveries["wide_runs"] == 0]
balls_faced = balls_faced.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()
balls_faced.columns = ["match_id", "inning", "batsman", "balls_faced"]
batsmen = batsmen.merge(balls_faced, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")

fours = deliveries[ deliveries["batsman_runs"] == 4]
sixes = deliveries[ deliveries["batsman_runs"] == 6]

fours_per_batsman = fours.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()
sixes_per_batsman = sixes.groupby(["match_id", "inning", "batsman"])["batsman_runs"].count().reset_index()

fours_per_batsman.columns = ["match_id", "inning", "batsman", "4s"]
sixes_per_batsman.columns = ["match_id", "inning", "batsman", "6s"]

batsmen = batsmen.merge(fours_per_batsman, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")
batsmen = batsmen.merge(sixes_per_batsman, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")
batsmen['SR'] = np.round(batsmen['batsman_runs'] / batsmen['balls_faced'] * 100, 2)

for col in ["batsman_runs", "4s", "6s", "balls_faced", "SR"]:
    batsmen[col] = batsmen[col].fillna(0)

dismissals = deliveries[ pd.notnull(deliveries["player_dismissed"])]
dismissals = dismissals[["match_id", "inning", "player_dismissed", "dismissal_kind", "fielder"]]
dismissals.rename(columns={"player_dismissed": "batsman"}, inplace=True)
batsmen = batsmen.merge(dismissals, left_on=["match_id", "inning", "batsman"], 
                        right_on=["match_id", "inning", "batsman"], how="left")

batsmen = matches[['id','season','city']].merge(batsmen, left_on = 'id', right_on = 'match_id', how = 'left').drop('id', axis = 1)
batsmen.head(2)

season       city  match_id  inning         batting_team      batsman  \
0    2017  Hyderabad         1       1  Sunrisers Hyderabad  BCJ Cutting   
1    2017  Hyderabad         1       1  Sunrisers Hyderabad    DA Warner   

   batsman_runs  balls_faced   4s   6s      SR dismissal_kind        fielder  
0            16          6.0  0.0  2.0  266.67            NaN            NaN  
1            14          8.0  2.0  1.0  175.00         caught  Mandeep Singh

In [6]:
raina=batsmen[batsmen['batsman'].str.contains("Raina")] 

raina1=raina[raina['batting_team'].str.contains("Chennai")] 

raina_home=raina1[raina1["city"]=="Chennai"]

len(raina_home)

raina_away=raina1[raina1["city"]!="Chennai"]

len(raina_away)

raina_home1=raina_home[np.logical_and(raina_home['season']!=2009,raina_home['season']!=2018)]

raina_away1=raina_away[np.logical_and(raina_away['season']!=2009,raina_away['season']!=2018)]
raina_away1=raina_away1[raina_away1["season"]!=2014]

raina_away1["season"].unique()

raina_home1["season"].unique()


raina_homes=raina_home1.sample(n = 20, random_state = 2) 

raina_home1

raina_aways=raina_away1.sample(n = 20, random_state = 2) 

raina_away1

len(raina_away1.columns)

import pandas as pd
from scipy import stats
ttest,p_val = stats.ttest_rel(raina_homes["batsman_runs"], raina_aways["batsman_runs"])
p_val = round(p_val,2)
print("p value: ",p_val)
if p_val<0.05:
    print("reject null hypothesis")
else:
    print("failed to reject null hypothesis")

p value:  0.26
failed to reject null hypothesis


In [7]:
print("p value: ",round(p_val,2))

p value:  0.26


In [8]:
answer =False

In [9]:
rohit=batsmen[batsmen['batsman'].str.contains("RG Sharma")] 

rohit

rohit1=rohit[rohit['batting_team'].str.contains("Mumbai")] 

rohit_home=rohit1[rohit1["city"]=="Mumbai"]

rohit_home

rohit_away=rohit1[rohit1["city"]!="Mumbai"]

rohit_away

len(rohit1)

rohit_away["season"].unique()

rohit_home["season"].unique()

rohit_homes=rohit_home.sample(n = 20, random_state = 2) 

rohit_homes

rohit_aways=rohit_away.sample(n = 20, random_state = 2) 

rohit_aways

import pandas as pd
from scipy import stats
ttest,pval1 = stats.ttest_rel(rohit_homes["SR"], rohit_aways["SR"])
pval1 = round(pval1,2)
print("pval 1: ",pval1)
if pval1<0.05:
    print("reject null hypothesis")
else:
    print("failed to reject null hypothesis")

pval 1:  0.3
failed to reject null hypothesis


In [10]:
print("p value: ",round(pval1,2))

p value:  0.3


In [11]:
answer1 =False

In [12]:
kohli=batsmen[batsmen['batsman'].str.contains("V Kohli")] 

kohli

kohli_home=kohli[kohli["city"]=="Bangalore"]

kohli_home1=kohli[kohli["city"]=="Bengaluru"]

len(kohli_home)

len(kohli_home1)

kohli_home2=pd.concat([kohli_home, kohli_home1])

kohli_home2

kohli_away=kohli[np.logical_and(kohli['city']!="Bangalore",kohli['city']!="Bengaluru")]

kohli_away

kohli_homes=kohli_home2.sample(n = 20, random_state = 2) 

kohli_homes

kohli_aways=kohli_away.sample(n = 20, random_state = 2) 

kohli_aways

import pandas as pd
from scipy import stats
ttest,pval2 = stats.ttest_rel(kohli_homes["6s"], kohli_aways["6s"])
pval2 = round(pval2,2)
print("pval 2: ",pval2)
if pval2<0.05:
    print("reject null hypothesis")
else:
    print("failed to reject null hypothesis")

pval 2:  0.04
reject null hypothesis


In [13]:
print("pval 2: ",round(pval2,2))

pval 2:  0.04


In [14]:
answer2 =True

In [15]:
from scipy import stats
import numpy as np
mean, sigma = np.mean(deliveries["over"]), np.std(deliveries["over"])


s=stats.norm.interval(0.999, loc=mean, scale=sigma/np.sqrt(15000))

s=(round(s[0]),round(s[1]))

s

len(deliveries)

batsman_grp = deliveries.groupby(["match_id", "inning", "batting_team", "batsman"])


batsmen = batsman_grp["batsman_runs"].sum().reset_index()


batsmen

overs = deliveries.groupby(["match_id", "inning"])


s = batsman_grp["over"].sum().reset_index()


s

deliveries.groupby(['match_id','inning'])['over'].sum()

deliveries1=deliveries[["match_id","over","inning"]]

deliveries1

deliveries2=deliveries1["match_id"].unique()

deliveries3=deliveries1["over"].unique()

deliveries3

uniqueValues = (deliveries['over'].append(deliveries['match_id'])).unique()


a=deliveries[['match_id', 'inning','over','total_runs']]

a=pd.DataFrame(a)
a

a=a.rename(columns={0: "match_id", 1: "inning",2:"over",3:"total_runs"})
a

overs1 = pd.DataFrame(a.groupby(['match_id','inning']).sum()['total_runs'])
overs1.head()

overs1=overs1.reset_index()
overs1.head(20)

matches.head(10)

overss=overs1[overs1["inning"]==1]

overss

overss=overss.sample(n = 200, random_state = 2) 

from scipy.stats import sem, t
from scipy import mean
confidence = 0.95
data = overss["total_runs"]

n = len(data)
m = mean(data)
std = np.std(data)
h = std * t.ppf((1 + confidence) / 2, n)

start = m - h
print(int(start))
end = m + h
print(int(end))

109
218


In [16]:
import numpy as np, scipy.stats as st

s=st.t.interval(0.95, len(overss["total_runs"])-1, loc=np.mean(overss["total_runs"]), scale=np.std(overss["total_runs"]))
s=(int(s[0]),int(s[1]))
s

(109, 218)

In [17]:
from scipy import stats
import numpy as np
mean, sigma = np.mean(overss["total_runs"]), np.std(overss["total_runs"])


s=stats.norm.interval(0.95, loc=mean, scale=sigma)
s=(int(s[0]),int(s[1]))
print("confidence intervalminimum: ",s[0])
print("confidence intervalmaximum: ",s[1])

confidence intervalminimum:  109
confidence intervalmaximum:  218


In [18]:
a,b= stats.norm.interval(0.95, loc= mean, scale= sigma)
confi_min = int(a)
confi_max = int(b)
###Assign the value here

print("confidence intervalminimum: ",    confi_min        )#Enter the minimum confidence interval value
print("confidence intervalmaximum: ",    confi_max         )#Enter the maximum confidence interval value

confidence intervalminimum:  109
confidence intervalmaximum:  218


In [19]:
matches1=matches.sample(n = 200, random_state = 2) 

In [20]:
# chi-squared test with similar proportions
df_chi=matches1
from scipy.stats import chi2_contingency
from scipy.stats import chi2
# contingency table
table = pd.crosstab(df_chi["toss_winner"],df_chi["winner"])
#print(table)
stat, p, dof, expected = chi2_contingency(table)
#print('dof=%d' % dof)
#print(expected)
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
p = round(p,2)
critical = round(critical,2)
stat = round(stat,2)
print('p value: ',p)
print('critical value: ',critical)
print('stat value: ',stat)
if stat >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
#print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')


p value:  0.0
critical value:  200.33
stat value:  910.66
Dependent (reject H0)
Dependent (reject H0)


In [21]:
print('p value: ',round(p,2))
print('critical value: ',round(critical,2))
print('stat value: ',round(stat,2))

p value:  0.0
critical value:  200.33
stat value:  910.66


In [22]:
answer3 =True
answer4 =True

In [23]:
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
# contingency table
table = pd.crosstab(df_chi["toss_decision"],df_chi["winner"])
#print(table)
stat1, p1, dof, expected = chi2_contingency(table)
#print('dof=%d' % dof)
#print(expected)
# interpret test-statistic
prob = 0.95
critical1 = chi2.ppf(prob, dof)
print('p value1: ',round(p1,2))
print('critical value1: ',round(critical1,2))
print('stat value1: ',round(stat1,2))
if stat1 >= critical1:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
#print('significance=%.3f, p=%.3f' % (alpha, p1))
if p1 <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')


p value1:  0.47
critical value1:  22.36
stat value1:  12.75
Independent (fail to reject H0)
Independent (fail to reject H0)


In [24]:
df6= matches[['toss_decision','winner']]
np.random.seed(2)
df6_sample= df6.sample(n=200, random_state=2)

df6_ct= pd.crosstab(df6_sample.toss_decision, df6_sample.winner, rownames=['toss_dec'], colnames= ['winner'])
###Assign the value here
stat1, p1, ddof, _ = stats.chi2_contingency(df6_ct)
critical1= stats.chi2.ppf(0.95,df=ddof)
p1= round(p1,2)
stat1= round(stat1, 2)
critical1= round(critical1, 2)
p1 = round(p1,2)
critical1 = round(critical1,2)
stat1 = round(stat1,2)

print('p value1: ',p1)
print('critical value1: ',critical1)
print('stat value1: ',stat1)
#p1= 0.39
#stat1= 12.75
#critical1= 21.03

p value1:  0.47
critical value1:  22.36
stat value1:  12.75


In [25]:
answer5 =False
answer6 =False

In [26]:
overss

a=deliveries[['match_id', 'inning','over','total_runs','batting_team']]

a=pd.DataFrame(a)
a

a=a.rename(columns={0: "match_id", 1: "inning",2:"over",3:"total_runs",4:"batting_team"})
a

overs1 = pd.DataFrame(a.groupby(['match_id','inning',"batting_team"]).sum()['total_runs'])
overs1.head()

overs1=overs1.reset_index()

0.05*100

overs1

0.467*100

overs1

overs2=overs1.loc[overs1['batting_team'].isin(['Chennai Super Kings','Kolkata Knight Riders','Mumbai Indians'])]

len(overs2)

overs3=overs2[overs2["inning"]==1]

len(overs3)

overs3=overs3.sample(n = 100, random_state = 2) 

df_anova = overs3
df_anova = df_anova[['batting_team','total_runs']]
grps = pd.unique(df_anova.batting_team.values)
d_data = {grp:df_anova['total_runs'][df_anova.batting_team == grp] for grp in grps}
F, pval3 = stats.f_oneway( d_data['Mumbai Indians'],d_data['Kolkata Knight Riders'],
                     d_data['Chennai Super Kings'])
pval3 = round(pval3,2)
print("p valueanova: ", pval3)
if p<0.05:
    print("reject null hypothesis")
else:
    print("failed to reject null hypothesis")

p valueanova:  0.37
reject null hypothesis


In [27]:
answer7 = False

In [28]:
overs4=overs2[overs2["inning"]==2]

len(overs4)

overs4=overs4.sample(n = 100, random_state = 2) 

df_anova = overs4
df_anova = df_anova[['batting_team','total_runs']]
grps = pd.unique(df_anova.batting_team.values)
d_data = {grp:df_anova['total_runs'][df_anova.batting_team == grp] for grp in grps}
F, pval4 = stats.f_oneway( d_data['Mumbai Indians'],d_data['Kolkata Knight Riders'],
                     d_data['Chennai Super Kings'])
pval4 = round(pval4,2)
print("p valueanova1: ", pval4)
if p1<0.05:
    print("reject null hypothesis")
else:
    print("failed to reject null hypothesis")


p valueanova1:  0.16
failed to reject null hypothesis


In [29]:
answer8 =False

In [30]:
with open("Raina.txt" ,"w") as file:
    file.write(str(p_val))
    file.write("\n")
    file.write(str(answer))
    
with open("Rohit.txt" ,"w") as file:
    file.write(str(pval1))
    file.write("\n")
    file.write(str(answer1))
    
with open("Kohli.txt" ,"w") as file:
    file.write(str(pval2))
    file.write("\n")
    file.write(str(answer2))
    
with open("First_Innings_Score.txt" ,"w") as file:
    file.write(str(confi_min))
    file.write("\n")
    file.write(str(confi_max))
    
with open("Toss_Winner.txt" ,"w") as file:
    file.write(str(p))
    file.write("\n")
    file.write(str(critical))
    file.write("\n")
    file.write(str(stat))
    file.write("\n")
    file.write(str(answer3))
    file.write("\n")
    file.write(str(answer4))
    
with open("Toss_Decision.txt" ,"w") as file:    
    file.write(str(p1))
    file.write("\n")
    file.write(str(critical1))
    file.write("\n")
    file.write(str(stat1))
    file.write("\n")
    file.write(str(answer5))
    file.write("\n")
    file.write(str(answer6))
    
with open("First_Innings.txt" ,"w") as file:    
    file.write(str(pval3))
    file.write("\n")
    file.write(str(answer7))
    
with open("Secon_Innings.txt" ,"w") as file:    
    file.write(str(pval4))
    file.write("\n")
    file.write(str(answer8))
    
    file.close()
    